In [0]:
%sql
SELECT * FROM bronze_layer.task_full;

DROP TABLE IF EXISTS bronze_layer.request_comment;


In [0]:
%python
import pandas as pd
from pyspark.sql import SparkSession

# Nombre de la tabla en Databricks
table_name = "bronze_layer.task_full"

# Leer la tabla como un Spark DataFrame
spark_df = spark.table(table_name)

# Convertir el Spark DataFrame en un DataFrame de pandas
df = spark_df.toPandas()

# Mostrar las primeras filas del DataFrame de pandas

df = df.loc[:, ['universal_request', 'comments']]
df = df.dropna(subset=['comments'])

df_separated_comments = df['comments'].str.split(r'(?=\d{4}-\d{2}-\d{2} \d{2}:\d{2}:\d{2} - )',expand=True).stack().reset_index(drop=True)

df_separated_comments = pd.DataFrame(df_separated_comments, columns=['comments'])

df_result = pd.concat([df['universal_request'].repeat(df['comments'].str.split(r'(?=\d{4}-\d{2}-\d{2} \d{2}:\d{2}:\d{2} - )').apply(len)).reset_index(drop=True), df_separated_comments],axis=1)
df_result = df_result[df_result['comments'].apply(lambda x: x.strip().lower() != '')]

df_separated_columns = df_result['comments'].str.split(' - ', expand=True).reset_index(drop=True)
df_separated_columns = df_separated_columns.iloc[:, :2]
df_separated_columns.columns=['timestamp','rest']

df_result2 = pd.concat([df_result.reset_index(drop=True), df_separated_columns],axis=1)

df_result2[['owner', 'comment']] = df_result2['rest'].str.split(r'\(Additional Comments\)', expand=True)

df_result2 = df_result2.sort_values(by=['universal_request', 'timestamp']).reset_index(drop=True)

df_result2['request_comment_id'] = df_result2.groupby('universal_request').cumcount() + 1

df_clean = df_result2[['universal_request','request_comment_id','timestamp','owner','comment']]
df_clean['comment'] = df_clean['comment'].str.strip()

spark_df = SparkSession.builder.getOrCreate().createDataFrame(df_clean)

# Nombre de la tabla donde se guardará
table_name = "bronze_layer.request_comment"

# Guardar el DataFrame de Spark en la tabla
spark_df.write.option("mergeSchema", "true").mode("overwrite").saveAsTable(table_name)

In [0]:
%sql 
SELECT * FROM bronze_layer.request_comment;

CREATE TABLE IF NOT EXISTS bronze_layer.brnz_request_comment AS
SELECT AX.*, 
current_timestamp() AS created_at_ts, 
current_timestamp() AS modified_at_ts, 
'CSV' AS datasource, 
'task_full.csv' AS input_file_name
FROM bronze_layer.request_comment AX;

SELECT * FROM bronze_layer.brnz_request_comment;